# Database pipeline for quantification of E.faecalis from metagenomic data

## What's in this notebook?

This notebook is a modification of `efaecalis.ipynb` to specifically use the following collection of isolates as a reference database:
1. The ~2000 isolates from European samples of _E.faecalis_
2. The ~350 E. faecalis isolates from the ELMC infant analysis (https://www.nature.com/articles/s41467-022-35178-5)
3. All other complete chromosomal assemblies from the Enterococcaceae family outside of _E. faecalis_.

*Note: This notebook is specifically meant for running the `infant-nt` dataset analysis and thus we also include isolates from that study.*

## Main Prerequisites

We require the pre-downloaded collection of ~2000 isolates, archived a TSV file specified by path specified by `EUROPEAN_ISOLATE_INDEX`.
We recommend spot-checking the isolates' fastQ files before archiving them into this index (e.g. using Kraken), and ensure that they truly are E. faecalis.

This notebook also requires the index of infant isolates, downloaded using the script `infant_nt/download_assemblies.sh`.

(Minor note: Note that we allow wildcards in the index file. For instance, you can split the contig multi-fasta into many files and specify a glob search; e.g. `<infant_id>/*.fasta`)

All indices above must be a TSV file, with columns ((*) is required):
1. Genus*
2. Species*
3. Strain name* (wrap with quotes if you must include whitespace)
4. Accession* (a holdover from "NCBI Accession", but it is really just an ID column that just needs to be unique per row.)
5. Assembly (For now, a metadata-only column, thus it is optional)
6. SeqPath*
7. ChromosomeLen (The length of the chromosome of this isolate/organism, or a best guess. This is metadata; convenient for post-hoc analyses for converting into sample-overall relabund.)
8. GFF (Metadata, pointing to the location of the corresponding annotation, if one exists)
   
The column ordering is not strict -- but the column header names ARE. We use pandas `pd.read_csv` to read in a DataFrame which uses the column header.

## Other prerequisites

Other than that, the standard prereqs apply:
We recommend using a `conda` environment for this notebook, with `ipywidgets` installed and updated. None of the operations of this notebook requires a GPU.
This notebook requires that the following software is installed.
- chronostrain (python>=3.10, the basic recipe `conda_basic.yml` or the full recipe `conda_full.yml`)
- primersearch (http://emboss.open-bio.org/, https://anaconda.org/bioconda/emboss)
- dashing2 (2023 Baker and Langmead: https://github.com/dnbaker/dashing2)

### Hardware requirements
 
None of the operations of this notebook requires a GPU. 
As of Aug 2023, we estimate that the contents of this notebook requires ~20 GB of hard disk space. 
At the time that we ran this pipeline, the catalog of non-isolate chromosomal assemblies totalled 14.3 GB, and isolates totalled 1.1GB.
Other files (such as the BLAST database, marker seeds and chronostrain-specific byproducts) totalled 5.4 GB, with a peak of ~28 GB when accounting for temporary files.

## File paths and environment variables

In [32]:
from pathlib import Path
import pandas as pd
import numpy as np
from typing import *

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

""" ============================================ EDIT THESE SETTINGS BASED ON USER'S CHOICE. ============================================ """
""" RefSeq catalog settings"""
TARGET_DIR = Path("/data/cctm/youn/infant_nt/database")  # the base directory for everything else.

# ========== for infant catalog. The infant index <INFANT_ISOLATE_INDEX> will be created using the infants <infant_id>/isolate_asemblies/metadata.tsv
INFANT_CATALOG_DIR = Path("/data/cctm/youn/infant_nt")
INFANT_ISOLATE_INDEX = TARGET_DIR / 'infant_isolates' / 'index.tsv'
# ========== European isolate catalog of E. faecalis
EUROPEAN_ISOLATE_INDEX = Path("/data/local/europe_efaecalis/index.tsv")
# ========== RefSeq index of Enterococcaceae (see efaecalis.ipynb. Download the index of chromosomal assemblies, then take the subset of all non-efaecalis genomes.)
REFSEQ_INDEX = Path("/data/cctm/youn/infant_nt/ref_genomes/index.tsv")
# filter out E.faecalis entries; we only want to use the isolates.
refseq_index_df = pd.read_csv(REFSEQ_INDEX, sep='\t')
refseq_index_df = refseq_index_df.loc[~((refseq_index_df['Genus'] == 'Enterococcus') & (refseq_index_df['Species'] == 'faecalis'))]
print(REFSEQ_INDEX)
REFSEQ_INDEX = REFSEQ_INDEX.parent / "enterococcaceae_index.tsv"
print(REFSEQ_INDEX)
refseq_index_df.to_csv(REFSEQ_INDEX, sep='\t', index=False)

""" PGAP ANNOTATION LOCATION """  # Refer to the cell which talks about pgap annotations on isolates
ISOLATE_PGAP_ANNOTATION_DIR=Path("/data/cctm/youn/infant_nt/isolate_pgap_annotations")

""" RefSeq BLAST database """
BLAST_DB_DIR = TARGET_DIR / "blast_db"
BLAST_DB_NAME = "Efcs_Europe_ELMC"  # Blast DB to create.

""" Marker seeds """
MARKER_SEED_DIR = TARGET_DIR / "marker_seeds"
MARKER_SEED_INDEX = MARKER_SEED_DIR / "marker_seed_index.tsv"

""" chronostrain-specific settings """
NUM_CORES = 8  # number of cores to use (e.g. for blastn)
MIN_PCT_IDTY = 75  # accept BLAST hits as markers above this threshold.
CHRONOSTRAIN_DB_DIR = TARGET_DIR / "chronostrain_files"  # The directory to use for chronostrain's database files.
CHRONOSTRAIN_TARGET_JSON = CHRONOSTRAIN_DB_DIR / "efaecalis.json"  # the desired final product.
CHRONOSTRAIN_TARGET_CLUSTERS = CHRONOSTRAIN_DB_DIR / "efaecalis.clusters.txt"  # the clustering file.
CHRONOSTRAIN_TARGET_CLUSTERS_99_99PCT = CHRONOSTRAIN_DB_DIR / "efaecalis.clusters_99_99pct.txt"  # Alternative clustering file at higher granularity
DASHING2_DIR = Path("/home/youn/work/bin")  # Directory that contains the dashing2 executable.


""" ============================================ DO NOT EDIT BELOW ============================================ """
""" environment variable extraction """
try:
    VARS_SET
except NameError:
    VARS_SET = True
    _cwd = %pwd
    _DB_HELPER_DIR = Path(_cwd).parent.parent / 'database'
    _start_path = %env PATH

# Work in database example directory, where all the helper scripts and settings.sh are.
%cd "$_DB_HELPER_DIR"
# Don't use GPU when importing jaxlib through chronostrain.
%env JAX_PLATFORM_NAME=cpu  
%env TARGET_TAXA=$TARGET_TAXA
%env NCBI_REFSEQ_DIR=$NCBI_REFSEQ_DIR
%env REFSEQ_INDEX=$REFSEQ_INDEX
# Need basic executables, such as "which" and "basename" (required by primersearch)
%env PATH=/usr/bin:$_start_path:$DASHING2_DIR

/data/cctm/youn/infant_nt/ref_genomes/index.tsv
/data/cctm/youn/infant_nt/ref_genomes/enterococcaceae_index.tsv
/home/youn/work/chronostrain/examples/database
env: JAX_PLATFORM_NAME=cpu
env: TARGET_TAXA=$TARGET_TAXA
env: NCBI_REFSEQ_DIR=$NCBI_REFSEQ_DIR
env: REFSEQ_INDEX=/data/cctm/youn/infant_nt/ref_genomes/enterococcaceae_index.tsv
env: PATH=/usr/bin:/home/youn/mambaforge/envs/chronostrain2/bin:/home/youn/work/bin


In [2]:
# === Ensure that these commands work.
print("checking EMBOSS primersearch.")
!primersearch --version

print("\nchecking dashing2.")
!dashing2 --version

print("\nchecking pgap.")
!pgap --version

checking EMBOSS primersearch.
EMBOSS:6.6.0.0

checking dashing2.
#Calling Dashing2 version v2.1.19 with command '/home/youn/work/chronostrain/examples/database/dashing2 --version'
dashing2 has several subcommands: sketch, cmp, wsketch, and contain.
Usage can be seen in those subcommands. (e.g., `dashing2 sketch -h`)

	sketch: converts FastX into k-mer sets/sketches, and sketches BigWig and BED files; also contains functionality from cmp, for one-step sketch and comparisons
This is probably the most common subcommand to use.

	cmp: compares previously sketched/decomposed k-mer sets and emits results. alias: dist

	contain: Takes a k-mer database (built with dashing2 sketch --save-kmers), then computes coverage for all k-mer references using input streams.
	wsketch: Takes a tuple of [1-3] input binary files [(u32 or u64), (float or double), (u32 or u64)] and performs weighted minhash sketching.
Three files are treated as Compressed Sparse Row (CSR)-format, where the third file contains i

## Recipe starts here.

In [3]:
# Prepare directories.
TARGET_DIR.mkdir(exist_ok=True, parents=True)
print(f"made {TARGET_DIR}")
BLAST_DB_DIR.mkdir(exist_ok=True, parents=True)
print(f"made {BLAST_DB_DIR}")
MARKER_SEED_DIR.mkdir(exist_ok=True, parents=True)
print(f"made {MARKER_SEED_DIR}")

made /data/cctm/youn/infant_nt/database
made /data/cctm/youn/infant_nt/database/blast_db
made /data/cctm/youn/infant_nt/database/marker_seeds


### Step 1 - include isolate assemblies

Include infant isolates to the database catalog.

*Note: This cell does nothing if `infant_nt/download_assemblies.sh` has not been run. It can be safely skipped if one does not want to include these isolates.*

In [9]:
# this file points to the output of infant_nt/download_assembly_catalog.sh.
from Bio import SeqIO

INFANT_ISOLATE_INDEX.parent.mkdir(exist_ok=True, parents=True)
infant_isolate_df_entries = []
for f in INFANT_CATALOG_DIR.glob("*/isolate_assemblies/metadata.tsv"):
    print("Found metadata in dir {}.".format(f.parent))

    # =============== Create a dataframe.
    df_dir = f.parent
    isolate_df = pd.read_csv(f, sep='\t')
    for _, row in isolate_df.iterrows():
        # Parse entries.
        infant_id = row['Participant']
        acc = row['Accession']
        genus = row['Genus']
        species = row['Species']
        timepoint = row['Timepoint']
        sample_id = row['SampleId']
        source_fasta_path = df_dir / Path(row['FastaPath']).name

        # Skip if not E. faecalis.
        if not (genus == 'Enterococcus' and species == 'faecalis'):
            continue

        # Extract the records.
        records = list(SeqIO.parse(source_fasta_path, format="fasta"))
        total_contig_len = sum(len(record.seq) for record in records)

        # Do nothing if no records are available.
        if len(records) == 0:
            continue

        # Add to the dataframe.
        infant_isolate_df_entries.append(
            (genus, species, f'{infant_id}_t:{timepoint}_s:{sample_id}', infant_id, timepoint, acc, acc, str(source_fasta_path), total_contig_len, 'None')
        )


# Create dataframe and save to file.
if len(infant_isolate_df_entries) > 0:
    infant_isolate_df = pd.DataFrame(
        infant_isolate_df_entries, 
        columns=['Genus', 'Species', 'Strain', 'Infant', 'T', 'Accession', 'Assembly', 'SeqPath', 'ChromosomeLen', 'GFF']
    ).astype(
        {
            'Genus': 'string',
            'Species': 'string',
            'Strain': 'string',
            'Infant': 'string',
            'T': 'string',
            'Accession': 'string',
            'Assembly': 'string',
            'SeqPath': 'string',
            'ChromosomeLen': 'int',
            'GFF': 'string'
        }
    )
    infant_isolate_df.to_csv(INFANT_ISOLATE_INDEX, sep='\t', index=False)

Found metadata in dir /data/cctm/youn/infant_nt/A00021_T1/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00021_T2/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00106_T1/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00106_T2/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/B00203_T1/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/B00203_T2/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/C02223_T1/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/C02223_T2/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00031/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00043/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00067/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00185/isolate_assemblies.
Found metadata in dir /data/cctm/youn/infant_nt/A00190/isolate_assemblies.
F

### Step 2: Build the marker seed catalog.

ChronoStrain only needs a FASTA file of marker seeds (one multi-fasta file per marker gene), and a single TSV file that catalogs them.
However, to get there, we need to take a few steps...

#### Step 2.1: Download MLST schema.

In [11]:
!python python_helpers/mlst_download.py -t "Enterococcus faecalis" -w "$MARKER_SEED_DIR"/mlst_schema -o "$MARKER_SEED_DIR"/mlst_seeds.tsv

Targeting 1 taxa using MLST scheme.
Fetching URL resource https://pubmlst.org/static/data/dbases.xml
Got a response of size 152.35 KB.
Schema type id: 
Handling locus gdh
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gdh/alleles_fasta
Got a response of size 76.36 KB.
Handling locus gyd
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gyd/alleles_fasta
Got a response of size 23.67 KB.
Handling locus pstS
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/pstS/alleles_fasta
Got a response of size 79.37 KB.
Handling locus gki
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gki/alleles_fasta
Got a response of size 57.64 KB.
Handling locus aroE
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/aroE/alleles_fasta
Got a response of size 65.53 KB.
Handling locus xpt
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/

#### Step 2.2: Non-standard genes

Here, we save some work and re-use the marker seeds discovered using PCR primers in the `database` example notebook (efaecalis.ipynb)

(The FASTA files representing primer-hits with lengths <150 were left out. We also took the first entry for each fasta file (due to the primer hits' overall similarity) except for fsrB which appears to have quite a bit of variability.)

In [13]:
!cp /mnt/e/efaecalis_db/marker_seeds/cbh.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsA.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsB.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsC.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsD.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsE.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsF.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsG.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsH.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsI.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsJ.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cpsK.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cylA.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/cylB.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/esp.fasta "$MARKER_SEED_DIR"/
!cp /mnt/e/efaecalis_db/marker_seeds/fsrB.fasta "$MARKER_SEED_DIR"/

with open(MARKER_SEED_DIR / "virulence_seeds.tsv", "wt") as metadata_tsv:
    print("{}\t{}\t{}".format("cbh", MARKER_SEED_DIR / "cbh.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsA", MARKER_SEED_DIR / "cpsA.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsB", MARKER_SEED_DIR / "cpsB.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsC", MARKER_SEED_DIR / "cpsC.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsD", MARKER_SEED_DIR / "cpsD.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsE", MARKER_SEED_DIR / "cpsE.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsF", MARKER_SEED_DIR / "cpsF.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsG", MARKER_SEED_DIR / "cpsG.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsH", MARKER_SEED_DIR / "cpsH.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsI", MARKER_SEED_DIR / "cpsI.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsJ", MARKER_SEED_DIR / "cpsJ.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cpsK", MARKER_SEED_DIR / "cpsK.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cylA", MARKER_SEED_DIR / "cylA.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("cylB", MARKER_SEED_DIR / "cylB.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("esp", MARKER_SEED_DIR / "esp.fasta", "POLYMORPHIC"), file=metadata_tsv)
    print("{}\t{}\t{}".format("fsrB", MARKER_SEED_DIR / "fsrB.fasta", "POLYMORPHIC"), file=metadata_tsv)

#### Step 2.3: Annotate isolates.

**Note 1: The ELMC isolates are grabbed via the `infant-nt` example (`download_assemblies.sh`)**

**Note 2: this step takes a long time; consider assigning the task to a dedicated compute cluster.** Alternatively, grab the pre-computed annotation GFF files from (Todo zenodo record URL)**, and place its contents into <INFANT_ISOLATE_INDEX.parent>, so that it matches the directory pattern `<ISOLATE_PGAP_ANNOTATION_DIR>/ <isolate_acc>`.

The cell has been included anyway, with the code commented out to demonstrate how it was done within the context of this notebook.

For clarity: The goal here is to run the `pgap` tool (Prokaryotic Genome Annotation Pipeline) from NCBI to annotate the genome of each isolate, via the command
`pgap -r -o <INFANT_ISOLATE_INDEX.parent>/annotations/<isolate_acc> -g <isolate_assembly_fasta> -s 'Enterococcus faecalis'`
where the fasta file `<isolate_assembly_fasta>` points to a multi-fasta file of the assembled contigs. 

The only catch is that `pgap` does not handle fasta records with malformed IDs, has a sequence of trailing N's on either ends, nor records with length < 200. Thus N's must be trimmed from each record and records of length < 200 (after trimming) must be removed.
([Reference: pgap github wiki](https://github.com/ncbi/pgap/wiki/Input-Files))
The isolate FASTA files have plenty of records that violate these rules, so some cleaning must be done.

In [51]:
#### Uncomment the code in this cell to run it within the notebok. This requires NCBI's pgap software.

# import os
# import shutil
# cwd = Path().resolve()

# print(f"Reading infant isolate catalog from {INFANT_CATALOG_DIR}/*/isolate_assemblies/metadata.tsv")
# for f in INFANT_CATALOG_DIR.glob("*/isolate_assemblies/metadata.tsv"):
#     infant_id = f.parent.parent.name
#     print(f"Handling isolates from {infant_id}")
    
#     # =============== Create a dataframe.
#     isolate_df = pd.read_csv(f, sep='\t')
#     for _, row in isolate_df.iterrows():
#         # Parse entries.
#         participant = row['Participant']
#         acc = row['Accession']
#         genus = row['Genus']
#         species = row['Species']
#         timepoint = row['Timepoint']
#         sample_id = row['SampleId']
#         source_fasta_path = Path(row['FastaPath'])

#         # Skip if not E. faecalis.
#         if not (genus == 'Enterococcus' and species == 'faecalis'):
#             continue

#         # setup
#         isolate_out_dir = INFANT_ISOLATE_INDEX.parent / "annotations" / acc
#         tmp_dir = isolate_out_dir.parent / '_tmp'
#         taxa_name = f'{genus} {species}'

#         isolate_out_dir.parent.mkdir(exist_ok=True, parents=True)
#         tmp_dir.mkdir(exist_ok=True)
        
#         # reformat fasta file, remove punctuations.
#         fixed_fasta_path = tmp_dir / f"{acc}.fasta"
#         with open(fixed_fasta_path, "wt") as f:
#             for r_idx, record in enumerate(SeqIO.parse(source_fasta_path, "fasta")):
#                 seq = record.seq.rstrip('N')
#                 if len(seq) < 200:
#                     l = len(record.seq)
#                     print(f"Skipping record {record.id} because it has length < 200 (len={l}) (possibly after stripping Ns)")
#                     continue
#                 record.id = f"{acc}:CONTIG_{r_idx}"
#                 record.seq = seq
#                 SeqIO.write([record], handle=f, format="fasta")

#         # run pgap.
#         os.chdir(tmp_dir)
#         print(f"[DIR = {tmp_dir}] pgap -r -o {isolate_out_dir} -g {fixed_fasta_path} -s '{taxa_name}'")
#         !pgap -r -o {isolate_out_dir} -g {fixed_fasta_path} -s '{taxa_name}' --quiet --no-self-update
        
#         # go back to working dir, and cleanup.
#         os.chdir(cwd)
#         shutil.rmtree(tmp_dir)

Next, parse the annotations and grab the genes. The cell below requires the package GFF package (https://github.com/chapmanb/bcbb/tree/master/gff)

In [14]:
import string
from BCBio import GFF
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
from tqdm.notebook import tqdm


# ================ Functions for appending gene sequences to fasta records
gene_paths = {}
def get_gene_path(gene_name):
    """
    Retrieves the gene path from the dist `gene_paths`, but with some pre-cleaning.
    """
    if gene_name in gene_paths:
        return gene_paths[gene_name]
    else:
        p = INFANT_ISOLATE_INDEX.parent / 'genes' / f'{gene_name}.fasta'
        if p.exists():
            p.unlink()  # clear the contents
        else:
            p.parent.mkdir(exist_ok=True, parents=True)
        gene_paths[gene_name] = p
        return p


def add_gene_seq(gene_name: str, gene_seq: Seq, gene_id: str, description: str = ""):
    """
    Adds a sequence (gene_seq) to a fasta file, where path is derived from get_gene_path.
    """
    gene_path = get_gene_path(gene_name)
    # this opens the fasta in "a" mode, but get_gene_path clears its contents to ensure that each time this code is run, it's a fresh start.
    with open(gene_path, "a") as f:  
        SeqIO.write(
            [SeqRecord(gene_seq, id=gene_id, name=gene_name, description=description)], 
            handle=f, 
            format='fasta'
        )


def remove_punctuations(s: str) -> str:
    """
    To ensure that a gene name can be translated into a POSIX-friendly file path, this function removes punctuations.
    """
    return s.translate(str.maketrans('', '', string.punctuation))
    

# ========================= Process the pgap results, using the above helpers.
df_entries = []

infant_isolate_df = pd.read_csv(INFANT_ISOLATE_INDEX, sep='\t')  # load the isolate catalog, which has metadata that we need

pbar = tqdm(infant_isolate_df.iterrows(), total=infant_isolate_df.shape[0])
for _, row in pbar:
    acc = row['Accession']
    pbar.set_postfix({"Isolate": acc})
    annot_subdir = ISOLATE_PGAP_ANNOTATION_DIR / acc
    gff_path = annot_subdir / "annot.gff"  # annotations file containing gene names and locations
    seq_path = annot_subdir / f"{acc}.fasta"  # the sequence fasta file to extract genes from
    if not gff_path.exists():
        print(f"annot.gff not found for {acc}. Did pgap run correctly?")
        continue

    seq_records = {record.id: record.seq for record in SeqIO.parse(seq_path, "fasta")}
    with open(gff_path, "rt") as gff_handle:
        for rec in GFF.parse(gff_handle, limit_info=dict(gff_type=['gene'])):
            # assume that the record is of the format (<isolate_acc>:CONTIG_<contig_index>), to be handled via the PGAP preprocessing. (see above cell)
            contig_idx = int(rec.id.split("_")[-1])
            
            for feature in rec.features:
                # find gene name; exclude putative genes (pgaptmp_*)
                gene_names = sorted([remove_punctuations(n) for n in feature.qualifiers['Name'] if not n.startswith('pgaptmp')])
                
                if len(gene_names) > 0:
                    gene_name = gene_names[0]  # if there are multiple candidate names, take the lexicographically first one.
                    loc = feature.location
                    gene_seq = loc.extract(seq_records[rec.id])

                    # Add count to dataframe
                    df_entries.append({
                        'Accession': acc,
                        'Contig': contig_idx,
                        'Gene': gene_name,
                        'Length': len(gene_seq),
                        'PositionLeft': int(loc.start),
                        'PositionRight': int(loc.end),
                        'PositiveStrand': loc.strand > 0
                    })

                    # write record to fasta
                    gene_id = f"{acc}:{gene_name}:{contig_idx}"
                    desc = f"Extracted from {acc} using pgap, position {str(loc)} of contig index {contig_idx}"
                    add_gene_seq(gene_name, gene_seq, gene_id, description=desc)
    
    del seq_records  # clean up

gene_annotations = pd.DataFrame(df_entries)
del df_entries  # clean up

gene_annotations.to_feather(INFANT_ISOLATE_INDEX.parent / "genes.feather")
display(gene_annotations)

  0%|          | 0/349 [00:00<?, ?it/s]

annot.gff not found for GCA_902159085. Did pgap run correctly?
annot.gff not found for GCA_902159895. Did pgap run correctly?
annot.gff not found for GCA_902159915. Did pgap run correctly?
annot.gff not found for GCA_902159945. Did pgap run correctly?
annot.gff not found for GCA_902160035. Did pgap run correctly?
annot.gff not found for GCA_902165355. Did pgap run correctly?
annot.gff not found for GCA_902165265. Did pgap run correctly?
annot.gff not found for GCA_902165295. Did pgap run correctly?
annot.gff not found for GCA_902165305. Did pgap run correctly?
annot.gff not found for GCA_902159845. Did pgap run correctly?
annot.gff not found for GCA_902159955. Did pgap run correctly?
annot.gff not found for GCA_902160045. Did pgap run correctly?
annot.gff not found for GCA_902163595. Did pgap run correctly?
annot.gff not found for GCA_902163605. Did pgap run correctly?
annot.gff not found for GCA_902165625. Did pgap run correctly?
annot.gff not found for GCA_902163975. Did pgap run cor

,Accession,Contig,Gene,Length,PositionLeft,PositionRight,PositiveStrand
0,GCA_902164875,0,rrf,116,94,210,True
1,GCA_902164875,0,bgsA,1005,482,1487,True
2,GCA_902164875,0,bgsB,1224,1533,2757,True
3,GCA_902164875,0,fabK,957,6278,7235,True
4,GCA_902164875,0,fabD,930,7267,8197,True
...,...,...,...,...,...,...,...
218638,GCA_902165805,8,sat4,543,3328,3871,True
218639,GCA_902165805,8,aph3IIIa,795,3963,4758,True
218640,GCA_902165805,9,cylR2,201,895,1096,False
218641,GCA_902165805,9,cylLL,207,1499,1706,True


Run multiple alignment. This step requires MAFFT.

In [22]:
import os
from tqdm.notebook import tqdm

gene_annotations = pd.read_feather(INFANT_ISOLATE_INDEX.parent / "genes.feather")
n_isolate_accs = len(pd.unique(gene_annotations['Accession']))
tol_frac = 0.5  # don't align genes found in fewer than this fraction of isolates

n_genes = len(pd.unique(gene_annotations['Gene']))
pbar = tqdm(gene_annotations.groupby("Gene"), total=n_genes, desc="Multiple Alignment")


for gene_name, gene_section in pbar:
    pbar.set_postfix({"Gene": gene_name})
    count = gene_section.shape[0]
    if count < tol_frac * n_isolate_accs:
        continue
    
    # run multiple alignment.
    gene_path = gene_paths[gene_name]
    aln_path = INFANT_ISOLATE_INDEX.parent / 'gene_alignments' / f'{gene_name}.aln.fasta'
    if not aln_path.parent.exists():
        aln_path.parent.mkdir(exist_ok=True, parents=True)
    if not aln_path.exists():
        os.system(f"mafft --nuc --quiet --thread 8 {gene_path} > {aln_path}")  # ">" overwrites any previous runs.

Multiple Alignment:   0%|          | 0/973 [00:00<?, ?it/s]

Parse the alignments. Pick the genes that best separate the isolates that are clustered together using only the ST genes + polymorphism/virulence islands.

But in order to do this, let's first construct the database with just MLST + virulence genes, to see how well these split the infant isolates:

In [17]:
!cat "$MARKER_SEED_DIR"/mlst_seeds.tsv > "$MARKER_SEED_DIR"/mlst_and_virulence.tsv
!cat "$MARKER_SEED_DIR"/virulence_seeds.tsv >> "$MARKER_SEED_DIR"/mlst_and_virulence.tsv


!mkdir -p "$BLAST_DB_DIR"
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR={CHRONOSTRAIN_DB_DIR} \
    CHRONOSTRAIN_LOG_INI={_cwd}/logging.ini \
    chronostrain -c chronostrain.ini \
        make-db \
        -m "$MARKER_SEED_DIR"/mlst_and_virulence.tsv \
        -r $INFANT_ISOLATE_INDEX \
        -b "infant_isolates_only" -bd $BLAST_DB_DIR \
        --min-pct-idty $MIN_PCT_IDTY \
        -o "$CHRONOSTRAIN_DB_DIR"/mlst_virulence_infant_isol.json \
        --threads $NUM_CORES

In [19]:
# Perform clustering
thresh = 0.998
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR="$CHRONOSTRAIN_DB_DIR" \
    CHRONOSTRAIN_LOG_INI={_cwd}/logging.ini \
    chronostrain -c chronostrain.ini \
      cluster-db \
      -i "$CHRONOSTRAIN_DB_DIR"/mlst_virulence_infant_isol.json \
      -o "$CHRONOSTRAIN_DB_DIR"/mlst_virulence_infant_isol.clusters.txt \
      --ident-threshold {thresh}

In [29]:
import itertools
from tqdm.notebook import tqdm


infant_isolate_df = pd.read_csv(INFANT_ISOLATE_INDEX, sep='\t')  # load the isolate catalog, which has metadata that we need
gene_annotations = pd.read_feather(INFANT_ISOLATE_INDEX.parent / "genes.feather")  # run this if previous cell already ran in a prior instantiation


# a function that will compute the necessary distance matrix.
def get_splitting_genes(infant_groupings: Dict[str, List[str]], infant_ordering: Dict[str, int]):
    """
    Uses each gene's distane matrix to determine a collection of genes which "distinguishes" them across infants.
    Refer to `load_gene_distance_matrix` for the details as to what the terms "distance" and "distinguishes" mean.
    """
    union_isolates = set()
    for _, isolate_list in infant_groupings.items():
        union_isolates = union_isolates.union(set(isolate_list))
        
    gene_subset = gene_annotations.loc[gene_annotations['Accession'].isin(union_isolates)]
    gene_names = []
    gene_nnzs = []
    for gene_name, gene_section in gene_subset.groupby("Gene"):
        # isolates_with_gene = set(gene_section['Accession'])
        # isolates_without_gene = this_isolates.difference(isolates_with_gene)

        try:
            d = load_gene_distance_matrix(gene_name, infant_groupings, infant_ordering, union_isolates)  # shape is (n_infants, n_infants)
        except FileNotFoundError:
            continue
        d_tril = d[np.tril_indices(n=len(infant_ordering), k=-1)]
        frac_nnz = np.sum(d_tril > 0) / len(d_tril)

        gene_names.append(gene_name)
        gene_nnzs.append(frac_nnz)
    ordering = np.argsort(gene_nnzs)[::-1]  # decreasing order

    return [
        gene_names[k]
        for k in ordering[:3]  # top three
    ]


def load_gene_distance_matrix(gene_name: str, infant_groupings: Dict[str, List[str]], infant_ordering: Dict[str, int], union_isolates: Set[str]) -> np.ndarray:
    """
    Uses the previously computed multiple alignments to create a distance matrix between infants.
    
    For each gene g, a matrix is computed:
    d[g](i, j) = min_{x \in isolates_i, y \in isolates_j} HAMMING_[gene=g](x, y)

    d[g](i, j) > 0 indicates that the gene g "cuts"/"distinguishes" the isolates of i versus the isolates of j, in the sense that 
    d[g] is a lower bound on the distance between any isolate of i and any isolate of j.
    """
    aln_path = INFANT_ISOLATE_INDEX.parent / 'gene_alignments' / f'{gene_name}.aln.fasta'
    if not aln_path.exists():
        raise FileNotFoundError(f"Multiple alignment for {gene_name} didn't run.")

    # Parse the multiple alignment fasta file.
    try:
        l_aln = len(next(iter(SeqIO.parse(aln_path, "fasta"))).seq)
    except StopIteration:
        # no alignments found in this file.
        raise Exception("For some reason, {} contained no alignments.".format(aln_path))
    aligned_seqs = {acc: Seq('-' * l_aln) for acc in union_isolates}  # the aligned strings per isolate.
    for record in SeqIO.parse(aln_path, "fasta"):
        record_acc, contig_idx, _ = record.id.split(":")  # third token is assumed to be the gene name (see the cell that runs pgap).
        aligned_seqs[record_acc] = record.seq

    # compute the distances.
    n = len(infant_ordering)
    d = np.zeros(shape=(n, n), dtype=int)
    for (i, i_idx), (j, j_idx) in itertools.combinations(
        infant_ordering.items(),
        r=2
    ):
        i_isolates = infant_groupings[i]
        j_isolates = infant_groupings[j]
        dist = np.min([hamming(aligned_seqs[x], aligned_seqs[y]) for x, y in itertools.product(i_isolates, j_isolates)])
        d[i_idx, j_idx] = dist
        d[j_idx, i_idx] = dist
    return d


def hamming(x: Seq, y: Seq) -> int:
    assert len(x) == len(y)
    return sum(1 for i, j in zip(x, y) if i != j)


# ================================ parse the clustering using the previous set of genes
prelim_cluster_df_entries = []
# let's open the output of "efaecalis.ipynb" notebook from the "database/complete_recipes" example.
# with open("/mnt/e/infant_nt/database/chronostrain_files_with_REFSEQ/efaecalis.clusters_without_isolate_genes.txt", "rt") as f:
with open(CHRONOSTRAIN_DB_DIR / "mlst_virulence_infant_isol.clusters.txt", "rt") as f:
    for line in f:
        if line.startswith("#"):
            continue
        tokens = line.rstrip().split("\t")
        rep = tokens[0]
        members = tokens[1].split(",")
        for member in members:
            if member.startswith("GCA"):  # only include isolates
                prelim_cluster_df_entries.append({'Accession': member, 'Cluster': rep})

prelim_cluster_df = pd.DataFrame(prelim_cluster_df_entries)
del prelim_cluster_df_entries
    

# ================================= go through each cluster one by one and pick gene(s) that splits it.
target_gene_set = set()
n_clusters = len(pd.unique(prelim_cluster_df['Cluster']))
pbar = tqdm(prelim_cluster_df.groupby("Cluster").count().sort_values('Accession', ascending=False).iterrows(), total=n_clusters)
for cluster_id, row in pbar:
    pbar.set_postfix({"Cluster": cluster_id, 'Genes_found': len(target_gene_set)})
    section = prelim_cluster_df.loc[prelim_cluster_df['Cluster'] == cluster_id]

    infant_groupings = {
        infant_id: [
            row['Accession']
            for _, row in infant_isolate_section.iterrows()
        ]
        for infant_id, infant_isolate_section in section.merge(infant_isolate_df, on='Accession').groupby("Infant")
    }
    if len(infant_groupings) < 2:
        continue

    infant_ordering = {infant_id: idx for idx, infant_id in enumerate(infant_groupings.keys())}
    target_genes = get_splitting_genes(infant_groupings, infant_ordering)
    target_gene_set = target_gene_set.union(set(target_genes))
    # break  # debug; delete to run the whole thing.
print("Target genes: {}".format(len(target_gene_set)))

# save this to disk.
with open(MARKER_SEED_DIR / "selected_isolate_splitting_genes.txt", "wt") as f:
    for g in target_gene_set:
        print(g, file=f)

  0%|          | 0/61 [00:00<?, ?it/s]

Target genes: 66


Finally, write these genes to a TSV file for including into the collection of seeds.

In [25]:
isolate_annotated_tsv = MARKER_SEED_DIR / "isolate_annotated_seeds.tsv"
with open(isolate_annotated_tsv, "wt") as metadata_tsv:
    for gene_name in target_gene_set:
        if gene_name == "galE" or gene_name == "ssb":  # these two genes have isolate assemblies with N's inside them. ChronoStrain doesn't handle these gracefully yet, so let's leave them out.
            continue
        gene_fasta_path = INFANT_ISOLATE_INDEX.parent / 'genes' / f'{gene_name}.fasta'
        print(
            "{}\t{}\t{}".format(
                gene_name, gene_fasta_path, f"ISOLATE_ANNOT"
            ), 
            file=metadata_tsv
        )
print(f"Wrote to {isolate_annotated_tsv}")

Wrote to /data/cctm/youn/infant_nt/database/marker_seeds/isolate_annotated_seeds.tsv


#### 3.5 Combine marker seed files.

In [27]:
!cat "$MARKER_SEED_DIR"/mlst_seeds.tsv > $MARKER_SEED_INDEX
!cat "$MARKER_SEED_DIR"/virulence_seeds.tsv >> $MARKER_SEED_INDEX
!cat "$MARKER_SEED_DIR"/isolate_annotated_seeds.tsv >> $MARKER_SEED_INDEX

print("Created Marker seed index: {}".format(MARKER_SEED_INDEX))
assert MARKER_SEED_INDEX.exists()

Created Marker seed index: /data/cctm/youn/infant_nt/database/marker_seeds/marker_seed_index.tsv


### Step 4: Run Chronostrain's make-db command.

By the end of the previous step, we have:

1) FASTA files for each gene, listing out seed sequence(s).
2) A TSV file (marker_seed_index.tsv) containing a list of gene names and the paths to each of these FASTA files.

Using these as inputs, we now construct the database files:
1) A JSON file of the strain records and their markers.
2) A TXT file of strain records clustered by similarity.

In [26]:
!mkdir -p "$BLAST_DB_DIR"
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR={CHRONOSTRAIN_DB_DIR} \
    CHRONOSTRAIN_LOG_INI={_cwd}/logging.ini \
    chronostrain -c chronostrain.ini \
        make-db \
        -m $MARKER_SEED_INDEX \
        -r $EUROPEAN_ISOLATE_INDEX \
        -r $REFSEQ_INDEX \
        -r $INFANT_ISOLATE_INDEX \
        -b $BLAST_DB_NAME -bd $BLAST_DB_DIR \
        --min-pct-idty $MIN_PCT_IDTY \
        -o $CHRONOSTRAIN_TARGET_JSON \
        --threads $NUM_CORES

In [50]:
# Perform clustering
thresh = 0.998
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR={CHRONOSTRAIN_DB_DIR} \
    CHRONOSTRAIN_LOG_INI={_cwd}/logging.ini \
    chronostrain -c chronostrain.ini \
      cluster-db \
      -i $CHRONOSTRAIN_TARGET_JSON \
      -o $CHRONOSTRAIN_TARGET_CLUSTERS \
      --ident-threshold {thresh}

In [ ]:
# Perform clustering
thresh = 0.9999
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR={CHRONOSTRAIN_DB_DIR} \
    CHRONOSTRAIN_LOG_INI={_cwd}/logging.ini \
    chronostrain -c chronostrain.ini \
      cluster-db \
      -i $CHRONOSTRAIN_TARGET_JSON \
      -o $CHRONOSTRAIN_TARGET_CLUSTERS_99_99PCT \
      --ident-threshold {thresh}

# OPTIONAL: Compute some statistics.

In [51]:
from chronostrain.database import JSONParser
src_db = JSONParser(
    entries_file=CHRONOSTRAIN_TARGET_JSON,
    data_dir=CHRONOSTRAIN_DB_DIR,
    marker_max_len=50000,
    force_refresh=False
).parse()

In [53]:
isolate_index = pd.read_csv(INFANT_ISOLATE_INDEX, sep='\t')
isolate_ids = set(isolate_index['Accession'])
n0 = sum(1 for s in src_db.all_strains() if not s.id in isolate_ids)
n1 = sum(1 for s in src_db.all_strains() if s.metadata.genus == 'Enterococcus' and s.metadata.species == 'faecalis' and not s.id.startswith("GCA"))
n2 = sum(1 for s in src_db.all_strains() if s.metadata.genus == 'Enterococcus' and s.metadata.species == 'faecalis' and s.id.startswith("GCA"))
print("# of total entries:", len(src_db.all_strains()))
print("# of non-infant isolate entries:", n0)
print("# of non-infant isolate E. faecalis entries:", n1)
print("# of infant isolate E. faecalis entries:", n2)


marker_lens = np.array([sum(len(m) for m in s.markers) for s in src_db.all_strains()])
genome_lens = np.array([s.metadata.total_len for s in src_db.all_strains()])
ratios = marker_lens / genome_lens
print("Database's E. faecalis marker fraction of genome: {} [mean]".format(np.mean(ratios)))

In [54]:
df_entries = []
with open(CHRONOSTRAIN_TARGET_CLUSTERS, "rt") as f:
    for line in f:
        if line.startswith("#"):
            continue
        tokens = line.rstrip().split("\t")
        rep = tokens[0]
        members = tokens[1].split(",")
        for member in members:
            # if member.startswith("GCA"):  # only include isolates
            df_entries.append({'Accession': member, 'Cluster': rep})
cluster_df = pd.DataFrame(df_entries)
del df_entries
print("# clusters = {}".format(len(pd.unique(cluster_df['Cluster']))))


n_efaecalis = 0
for cluster_id in pd.unique(cluster_df['Cluster']):
    s = src_db.get_strain(cluster_id)
    if s.metadata.genus == 'Enterococcus' and s.metadata.species == 'faecalis':
        n_efaecalis += 1
print("# efaecalis clusters = {}".format(n_efaecalis))


n_infant_clusters = len(pd.unique(cluster_df.loc[cluster_df['Accession'].str.startswith("GCA"), "Cluster"]))
print("# efaecalis clusters with infant isolates = {}".format(n_infant_clusters))


n_infants_per_cluster = []
for cluster_id, section in cluster_df.loc[cluster_df['Accession'].str.startswith("GCA")].merge(infant_isolate_df, on='Accession').groupby("Cluster"):
    infant_ids = list(pd.unique(section['Infant']))
    n_infants_per_cluster.append(len(infant_ids))
    # print(cluster_id, "->", infant_ids)
print("# src infants per cluster containing some isolate [min={}, max={}, mean={}, median={}]".format(
    np.min(n_infants_per_cluster),
    np.max(n_infants_per_cluster),
    np.mean(n_infants_per_cluster),
    np.median(n_infants_per_cluster)
))

# [DEPRECATED CODE BELOW] mSWEEP database (poppunk clustering)

To obtain PopPUNK clustering, it was run with the following commands/settings:

```
poppunk --create-db --output EFaec --r-files input.tsv
poppunk --fit-model dbscan --ref-db EFaec --output dbscan
poppunk --fit-model refine --ref-db EFaec --model-dir dbscan --output refine --max-a-dist 0.9 --max-pi-dist 0.9
```

where `input.tsv` is the table of ~2000 european E.faecalis isolates, plus the ~350 infant E.faecalis isolates from ELMC.

In [37]:
poppunk_clust = pd.read_csv("/mnt/e/infant_nt/database/mgems/ref_dir/Efaecalis/ref_clu.tsv", sep="\t").rename(columns={"id": "Accession", "cluster": "Cluster"})
poppunk_clust

In [38]:
print("# efaecalis clusters = {}".format(len(pd.unique(poppunk_clust['Cluster']))))


n_infant_clusters = len(pd.unique(poppunk_clust.loc[poppunk_clust['Accession'].str.startswith("GCA"), "Cluster"]))
print("# efaecalis clusters with infant isolates = {}".format(n_infant_clusters))


n_infants_per_cluster = []
for cluster_id, section in poppunk_clust.loc[poppunk_clust['Accession'].str.startswith("GCA")].merge(infant_isolate_df, on='Accession').groupby("Cluster"):
    infant_ids = list(pd.unique(section['Infant']))
    n_infants_per_cluster.append(len(infant_ids))
    # print(cluster_id, "->", infant_ids)

print("# src infants per cluster containing some isolate [min={}, max={}, mean={}, median={}]".format(
    np.min(n_infants_per_cluster),
    np.max(n_infants_per_cluster),
    np.mean(n_infants_per_cluster),
    np.median(n_infants_per_cluster)
))